## Rank : 51                                                        

## Score : 0.7106616939

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predicting-employee-attrition/sample_submission_znWiLZ4.csv
/kaggle/input/predicting-employee-attrition/test_hXY9mYw.csv
/kaggle/input/predicting-employee-attrition/train_MpHjUjU.csv


### Importing relevant libraries

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action = 'ignore')

### Reading the data

In [3]:
train = pd.read_csv('../input/predicting-employee-attrition/train_MpHjUjU.csv')
test = pd.read_csv('../input/predicting-employee-attrition/test_hXY9mYw.csv')
sample = pd.read_csv('../input/predicting-employee-attrition/sample_submission_znWiLZ4.csv')

### Understanding the data

In [4]:
train.head()

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,2381060,2
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,-665480,2
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,2,2,0,1
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,2,2,0,1


In [5]:
test.head()

,Emp_ID
0,394
1,173
2,1090
3,840
4,308


In [6]:
sample.head()

,Emp_ID,Target
0,394,0
1,173,0
2,1090,0
3,840,0
4,308,0


In [7]:
train.shape, test.shape

((19104, 13), (741, 1))

In [8]:
train.isnull().sum()

MMM-YY                      0
Emp_ID                      0
Age                         0
Gender                      0
City                        0
Education_Level             0
Salary                      0
Dateofjoining               0
LastWorkingDate         17488
Joining Designation         0
Designation                 0
Total Business Value        0
Quarterly Rating            0
dtype: int64

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19104 entries, 0 to 19103
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   MMM-YY                19104 non-null  object
 1   Emp_ID                19104 non-null  int64 
 2   Age                   19104 non-null  int64 
 3   Gender                19104 non-null  object
 4   City                  19104 non-null  object
 5   Education_Level       19104 non-null  object
 6   Salary                19104 non-null  int64 
 7   Dateofjoining         19104 non-null  object
 8   LastWorkingDate       1616 non-null   object
 9   Joining Designation   19104 non-null  int64 
 10  Designation           19104 non-null  int64 
 11  Total Business Value  19104 non-null  int64 
 12  Quarterly Rating      19104 non-null  int64 
dtypes: int64(7), object(6)
memory usage: 1.9+ MB


### Creating test dataset

In [10]:
new_train = train.drop_duplicates(subset='Emp_ID', keep='last', ignore_index=True)

In [11]:
new_train.head()

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating
0,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2
1,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,2,2,0,1
2,2017-04-01,4,43,Male,C13,Master,65603,2016-12-07,2017-04-27,2,2,0,1
3,2016-03-01,5,29,Male,C9,College,46368,2016-01-09,2016-03-07,1,1,0,1
4,2017-12-01,6,31,Female,C11,Bachelor,78728,2017-07-31,NaN,3,3,0,2


In [12]:
test = pd.merge(test, new_train, how='left',on='Emp_ID')

In [13]:
test.head()

,Emp_ID,MMM-YY,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating
0,394,2017-12-01,34,Female,C20,Master,97722,2011-06-06,NaN,2,4,2701750,3
1,173,2017-12-01,39,Male,C28,College,56174,2011-06-13,NaN,1,3,706010,3
2,1090,2017-12-01,39,Male,C13,College,96750,2011-06-27,NaN,2,4,1518520,2
3,840,2017-12-01,40,Female,C8,College,88813,2011-07-11,NaN,1,4,1151820,2
4,308,2017-12-01,32,Male,C5,Master,188418,2011-08-15,NaN,2,5,3772910,2


### Creating the target variable

In [14]:
train['Target'] = np.where(train['LastWorkingDate'].isnull(), 0, 1)

In [15]:
train.head()

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating,Target
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,2381060,2,0
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,-665480,2,0
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2,1
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,2,2,0,1,0
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,2,2,0,1,0


### Dropping rows of test data from training dataset

In [16]:
test['Emp_ID'].unique()

array([ 394,  173, 1090,  840,  308, 1864, 1606,  954, 2422, 1841, 1484,
       2727, 1031, 2137, 2568, 1745, 2668, 2617, 2070, 1446, 1043, 2561,
        945, 2517, 1039, 2594,  115,  870,  329,  973, 1892, 2740, 1093,
        693, 1817, 1753, 2420, 1655, 1573, 2365, 1623, 2625, 1335,  450,
       2087, 2576, 2771, 1549, 1657, 1957, 1662, 1588, 2615, 2027, 2714,
        556,  377,  605,  791, 1147,   68, 2784, 1181, 1782, 2299, 1238,
        402, 1109,  990,  425,  569, 2454,  926,  391, 2110, 1875,  117,
       1714, 1106,  422,  150, 2179, 2684,  762,  718, 1694, 1051, 2104,
       1225,   63,  275, 1733, 2541,  140, 1037,  407, 1770,  946,  604,
        893, 2200, 2114, 1243, 2076, 2597,   60,  662,  765,  199, 1526,
       1486, 1179, 1161, 2259, 1191, 1811, 1510,  434, 1918,  753,   25,
       1414, 1426,  716, 2283, 1840, 1678, 1859, 2381, 2008, 1050,  581,
       2289,  713, 1924, 2257, 1504, 1670, 2551, 1874,   26, 2308, 1259,
         77,  568, 1370,  704, 2494, 1077,  592, 23

In [17]:
df = train[train['Emp_ID'].isin([394,  173, 1090,  840,  308, 1864, 1606,  954, 2422, 1841, 1484,
       2727, 1031, 2137, 2568, 1745, 2668, 2617, 2070, 1446, 1043, 2561,
        945, 2517, 1039, 2594,  115,  870,  329,  973, 1892, 2740, 1093,
        693, 1817, 1753, 2420, 1655, 1573, 2365, 1623, 2625, 1335,  450,
       2087, 2576, 2771, 1549, 1657, 1957, 1662, 1588, 2615, 2027, 2714,
        556,  377,  605,  791, 1147,   68, 2784, 1181, 1782, 2299, 1238,
        402, 1109,  990,  425,  569, 2454,  926,  391, 2110, 1875,  117,
       1714, 1106,  422,  150, 2179, 2684,  762,  718, 1694, 1051, 2104,
       1225,   63,  275, 1733, 2541,  140, 1037,  407, 1770,  946,  604,
        893, 2200, 2114, 1243, 2076, 2597,   60,  662,  765,  199, 1526,
       1486, 1179, 1161, 2259, 1191, 1811, 1510,  434, 1918,  753,   25,
       1414, 1426,  716, 2283, 1840, 1678, 1859, 2381, 2008, 1050,  581,
       2289,  713, 1924, 2257, 1504, 1670, 2551, 1874,   26, 2308, 1259,
         77,  568, 1370,  704, 2494, 1077,  592, 2390,  832,  904, 2567,
       1274, 2543,  296, 1374, 1852,  252,  888, 1234,  112, 2399, 1265,
       2166,  320, 1968, 2690, 2228,   56,  723,  560,  580, 1679, 1783,
        371, 2546, 2505, 2224, 1271,  213, 1936, 1642, 1213,   78,  368,
        943, 1165, 1432, 2223, 2407, 1316, 1139,  789, 2552, 2670, 2028,
         67, 2055, 2198, 2272, 2210, 2398,  272, 2754, 2473, 1327,  359,
       2160, 1563, 1206, 1858,  516,   41, 1382, 1405, 2730,  772,   54,
       1222,  697,  453, 1334,  992,  682,  766, 1283, 2148, 2729, 1668,
       1290, 1013,  906, 2357,   87, 2533, 1532, 1977,  565, 1598,  400,
       2109,  168,  956, 2417, 1276,  734,  550,  532, 2708, 2050,  358,
       1177, 1267, 1982,  933,  538, 1057, 2453,  188, 2241,  552, 1119,
       2382, 1284, 2480, 1658,  582, 2258,  444, 1633, 1958, 2334, 2633,
        388,  537, 1579, 2193, 1263, 2318,  351,  835,  493,  341,  191,
       1929,  991,  828,  234, 1869,  570, 2012,  323, 1249, 1989, 1830,
       2496,  446, 1475, 1972,  639, 1888,  710, 2639, 2014,  848, 2781,
       2199,  357,  494, 1178, 2611, 2728, 1149,  521,  375, 2763,  192,
        658,  109, 2052, 1752, 2483, 2386, 2205, 2347, 2569,  644, 2449,
       1230, 1652,  698, 1200, 1078, 1592,  210, 1144, 1980, 2096,  752,
        258, 2610, 2217,  362,  619, 1527,  999,  431, 1793,  501,  483,
       1605, 1836, 2464, 1914,  271, 1805, 2446, 1741, 2078,   45,  721,
        265,  806,  587, 1884,  617, 1070, 2082,  137, 2788, 1447,  960,
       2208,  865,  472,  441, 2699,  740, 1837,  782, 2715, 2344, 2169,
        889,  233, 2497, 1495,  509,  981,  276,   31, 2511, 1801,  595,
       2178,  324,   52, 1599, 1393,  643, 1505,  129, 1956, 2276, 1419,
       2664, 1372,  785, 1097, 1831,  409,  289,  673,  225, 1776, 1345,
       2322,  648, 1399, 2761, 1231,  571, 1962, 1594, 2705, 2498, 2090,
          6, 1073, 1433, 2439, 1666, 1118, 1087, 1871,  345,  597, 1214,
       2587, 1099,  959, 2672, 1676,  583, 1032, 1341, 2089, 1764, 1116,
       1637, 1514, 2423,  458,  376, 1169, 2686, 2254, 1054,  524,  122,
       2752,  988, 1015,  218, 2345,   39,  737, 2424, 2188, 1428, 2646,
       2331,  642, 2135, 1198,  559,  875,  982, 1114,  243,  548, 1159,
       1489, 2566, 2097,  678,  712,  681,  801, 1981,  273,  547, 1468,
       1325, 1518,  902, 1712, 2311,  237, 1268, 2537, 2378,  104,  510,
       1828, 2277, 2139, 1401,  620,  348, 2230,  531, 1707, 1696, 2565,
        936, 1056, 2440, 1724, 1815, 2202, 1634, 2043, 1807, 2654,  135,
       1066, 1586,  546,  594,  449,  415, 2058, 2418,  331, 2500, 2343,
       1965,  114, 1416,  591, 2426, 1125, 1438, 1322, 1286, 1434,  867,
       2645, 2640,  385, 1725,  427,  499, 1381, 1550, 1804,  543,   80,
       1710, 1190, 1115,   97, 1019,  462, 1873, 2764, 2775, 2260,   47,
       1014, 1622,  206, 1199,   85, 2018,  281,  263,  403, 2395, 1294,
        134,  646,  885, 1462, 2153,   14,   72,  382, 2722,  754, 1766,
        420,  294, 1839, 2032,  459,  963, 1775, 1999,  366,  793, 1515,
       2191, 2538, 1723,  202, 1626, 1153, 1379,  769,  417,  498, 1802,
        181, 2307, 1618,    2, 2279, 1351,  283, 2469,  650, 2627, 2348,
       2515,  412,  751, 2678, 1201, 1529,  457, 1026,  814,  335, 1791,
       1625, 1501,  763, 2635, 1107, 2411,  534,  952, 1412,   62, 2770,
       2140, 2778, 2428, 2571, 1324, 2239, 1227, 1448, 2290, 1209, 2550,
       1358,  540,  306,  810, 2631, 1740, 2030,  834, 1632, 2298,   11,
       1521, 1650, 1273, 1619, 1157,  398, 1568,  148,  664,  310,  825,
       1955, 1812, 1194, 1880,  599,  816,  508,  309, 1328, 1162,  733,
       2039,  378, 2360,  824, 1708,  511, 2060,  802, 1925,  297,  715,
       2233, 1195, 1145,  748, 1971, 1029,  288, 2521,  575, 2261,  561,
        274,  913,  622,  205, 1210,  711, 2088, 2522, 1188, 1111, 2753,
        638,  440, 2660, 2487, 2196, 2725, 2314,  819, 2189, 2251, 2134,
       2255,  448, 1644,  624])==False]

In [18]:
df.shape, test.shape

((10516, 14), (741, 13))

In [19]:
df['Target'].value_counts()

0    8900
1    1616
Name: Target, dtype: int64

### Exploratory Data Analysis

In [20]:
df[df['LastWorkingDate'].isnull() == False]

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating,Target
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2,1
9,2017-04-01,4,43,Male,C13,Master,65603,2016-12-07,2017-04-27,2,2,0,1,1
12,2016-03-01,5,29,Male,C9,College,46368,2016-01-09,2016-03-07,1,1,0,1,1
20,2017-11-01,8,34,Male,C2,College,70656,2017-09-19,2017-11-15,3,3,0,1,1
27,2016-12-01,12,35,Male,C23,Master,28116,2016-06-29,2016-12-21,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19039,2017-02-01,2779,28,Male,C26,College,95133,2017-01-26,2017-02-14,3,3,0,1,1
19054,2016-08-01,2782,26,Male,C19,Bachelor,29582,2016-05-16,2016-08-16,1,1,0,1,1
19081,2017-10-01,2785,34,Female,C9,College,12105,2017-08-28,2017-10-28,1,1,0,1,1
19090,2016-09-01,2786,45,Male,C19,College,35370,2015-07-31,2016-09-22,2,2,0,1,1


In [21]:
df['Gender'].loc[df['LastWorkingDate'].isnull() == False].value_counts()

Male      948
Female    668
Name: Gender, dtype: int64

In [22]:
df['Education_Level'].loc[df['LastWorkingDate'].isnull() == False].value_counts()

Master      547
College     542
Bachelor    527
Name: Education_Level, dtype: int64

In [23]:
df['Quarterly Rating'].loc[df['LastWorkingDate'].isnull() == False].value_counts()

1    1432
2     146
3      28
4      10
Name: Quarterly Rating, dtype: int64

In [24]:
df['Joining Designation'].loc[df['LastWorkingDate'].isnull() == False].value_counts()

1    752
2    560
3    274
4     22
5      8
Name: Joining Designation, dtype: int64

In [25]:
df['Designation'].loc[df['LastWorkingDate'].isnull() == False].value_counts()

2    600
1    596
3    337
4     70
5     13
Name: Designation, dtype: int64

In [26]:
df['Total Business Value'].loc[df['LastWorkingDate'].isnull() == False].value_counts()

 0         1392
 200000       7
 350000       7
 500000       4
 400000       3
           ... 
-141140       1
 197200       1
 990000       1
 193430       1
 162120       1
Name: Total Business Value, Length: 205, dtype: int64

Following things has been observed about the employee who has left the organisation:-
* Total business value of the employee is 0.
* Quarterly rating has been decreased to 1 or has been 1 from a long time.
* Joining designation and designation at the time of leaving the company was same. That is employee hasn't got the promotion.
* Mostly employee with designation 1 and 2 has left the organisation the most.
* Male employees has more attrition level as compared to female employees.

### Feature Engineering

In [27]:
df['MMM-YY'] = pd.to_datetime(df['MMM-YY']) 
df['Dateofjoining'] = pd.to_datetime(df['Dateofjoining'])

test['MMM-YY'] = pd.to_datetime(test['MMM-YY']) 
test['Dateofjoining'] = pd.to_datetime(test['Dateofjoining'])

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10516 entries, 0 to 19096
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   MMM-YY                10516 non-null  datetime64[ns]
 1   Emp_ID                10516 non-null  int64         
 2   Age                   10516 non-null  int64         
 3   Gender                10516 non-null  object        
 4   City                  10516 non-null  object        
 5   Education_Level       10516 non-null  object        
 6   Salary                10516 non-null  int64         
 7   Dateofjoining         10516 non-null  datetime64[ns]
 8   LastWorkingDate       1616 non-null   object        
 9   Joining Designation   10516 non-null  int64         
 10  Designation           10516 non-null  int64         
 11  Total Business Value  10516 non-null  int64         
 12  Quarterly Rating      10516 non-null  int64         
 13  Target          

In [29]:
df['experience'] = df['MMM-YY'] - df['Dateofjoining']

test['experience'] = test['MMM-YY'] - test['Dateofjoining']

In [30]:
df.head()

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating,Target,experience
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,2381060,2,0,8 days
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,-665480,2,0,39 days
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2,1,68 days
5,2016-12-01,4,43,Male,C13,Master,65603,2016-12-07,NaN,2,2,0,1,0,-6 days
6,2017-01-01,4,43,Male,C13,Master,65603,2016-12-07,NaN,2,2,0,1,0,25 days


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10516 entries, 0 to 19096
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype          
---  ------                --------------  -----          
 0   MMM-YY                10516 non-null  datetime64[ns] 
 1   Emp_ID                10516 non-null  int64          
 2   Age                   10516 non-null  int64          
 3   Gender                10516 non-null  object         
 4   City                  10516 non-null  object         
 5   Education_Level       10516 non-null  object         
 6   Salary                10516 non-null  int64          
 7   Dateofjoining         10516 non-null  datetime64[ns] 
 8   LastWorkingDate       1616 non-null   object         
 9   Joining Designation   10516 non-null  int64          
 10  Designation           10516 non-null  int64          
 11  Total Business Value  10516 non-null  int64          
 12  Quarterly Rating      10516 non-null  int64          
 13  T

In [32]:
df['experience'] = df['experience'].astype(str)

test['experience'] = test['experience'].astype(str)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10516 entries, 0 to 19096
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   MMM-YY                10516 non-null  datetime64[ns]
 1   Emp_ID                10516 non-null  int64         
 2   Age                   10516 non-null  int64         
 3   Gender                10516 non-null  object        
 4   City                  10516 non-null  object        
 5   Education_Level       10516 non-null  object        
 6   Salary                10516 non-null  int64         
 7   Dateofjoining         10516 non-null  datetime64[ns]
 8   LastWorkingDate       1616 non-null   object        
 9   Joining Designation   10516 non-null  int64         
 10  Designation           10516 non-null  int64         
 11  Total Business Value  10516 non-null  int64         
 12  Quarterly Rating      10516 non-null  int64         
 13  Target          

In [34]:
df['experience_in_days'] = df.experience.str.split(' ').apply(lambda x: x[0])

test['experience_in_days'] = test.experience.str.split(' ').apply(lambda x: x[0])

In [35]:
df.head()

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating,Target,experience,experience_in_days
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,2381060,2,0,8 days,8
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,-665480,2,0,39 days,39
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2,1,68 days,68
5,2016-12-01,4,43,Male,C13,Master,65603,2016-12-07,NaN,2,2,0,1,0,-6 days,-6
6,2017-01-01,4,43,Male,C13,Master,65603,2016-12-07,NaN,2,2,0,1,0,25 days,25


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10516 entries, 0 to 19096
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   MMM-YY                10516 non-null  datetime64[ns]
 1   Emp_ID                10516 non-null  int64         
 2   Age                   10516 non-null  int64         
 3   Gender                10516 non-null  object        
 4   City                  10516 non-null  object        
 5   Education_Level       10516 non-null  object        
 6   Salary                10516 non-null  int64         
 7   Dateofjoining         10516 non-null  datetime64[ns]
 8   LastWorkingDate       1616 non-null   object        
 9   Joining Designation   10516 non-null  int64         
 10  Designation           10516 non-null  int64         
 11  Total Business Value  10516 non-null  int64         
 12  Quarterly Rating      10516 non-null  int64         
 13  Target          

In [37]:
df['experience_in_days'] = df['experience_in_days'].astype(int)

test['experience_in_days'] = test['experience_in_days'].astype(int)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10516 entries, 0 to 19096
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   MMM-YY                10516 non-null  datetime64[ns]
 1   Emp_ID                10516 non-null  int64         
 2   Age                   10516 non-null  int64         
 3   Gender                10516 non-null  object        
 4   City                  10516 non-null  object        
 5   Education_Level       10516 non-null  object        
 6   Salary                10516 non-null  int64         
 7   Dateofjoining         10516 non-null  datetime64[ns]
 8   LastWorkingDate       1616 non-null   object        
 9   Joining Designation   10516 non-null  int64         
 10  Designation           10516 non-null  int64         
 11  Total Business Value  10516 non-null  int64         
 12  Quarterly Rating      10516 non-null  int64         
 13  Target          

In [39]:
df.shape, test.shape

((10516, 16), (741, 15))

### Dropping variables

In [40]:
df.drop(columns = ['experience','LastWorkingDate','Dateofjoining','MMM-YY'], axis=1, inplace=True)

test.drop(columns = ['experience','LastWorkingDate','Dateofjoining','MMM-YY'], axis=1, inplace=True)

In [41]:
df.shape, test.shape

((10516, 12), (741, 11))

### Dummy Encoding

In [42]:
df.select_dtypes(include='object').columns

Index(['Gender', 'City', 'Education_Level'], dtype='object')

In [43]:
test.select_dtypes(include='object').columns

Index(['Gender', 'City', 'Education_Level'], dtype='object')

In [44]:
df = pd.get_dummies(df, columns=['Gender', 'City', 'Education_Level'])
test = pd.get_dummies(test, columns=['Gender', 'City', 'Education_Level'])

### Assigning Target Variable

In [45]:
X = df.drop(columns = ['Target'], axis = 1)
y = df['Target']

### Scaling

In [46]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
test = scaler.transform(test)

### Train test split

In [47]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)

### Model Building

In [48]:
model = LogisticRegression(class_weight='balanced')
model.fit(X_train,y_train)
y_pred = model.predict(X_train)
score_train= f1_score(y_train,y_pred)


y_pred1 = model.predict(X_valid)
score_test= f1_score(y_valid,y_pred1)


print('Train F1 score:',score_train)
print('Valid F1 score:',score_test)

Train F1 score: 0.37985023825731795
Valid F1 score: 0.3934890304317056


### Prediction on test data

In [49]:
predictions = model.predict(test)
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,

### Submission

In [50]:
sample['Target'] = predictions

In [51]:
sample['Target'].value_counts()

0    466
1    275
Name: Target, dtype: int64